## Vlasov-Maxwell Landau Damping demonstration

In [1]:
import numpy as np

# Define the number of Hermite modes
Nn = Nm = Np = 10

n_idx = []

for n in range(Nn):
  for m in range(Nm):
    for p in range(Np):
      n_idx.append((n, m, p))

n_data = [(Nn, Nm, Np), n_idx]

def get_idx(s, n, m, p, N):
  if n < 0 or m < 0 or p < 0:
    return -1
  Nn, Nm, Np = N
  return s*Nn*Nm*Np + n*Nm*Np + m*Np + p

dim = 2*Nn*Nm*Np + 3
A = np.zeros((dim, dim), dtype = 'complex_')

k = (1,1,1)

qs = (-1,1)

### Create $L_1$ operator

In [2]:
def L1(A, n_data, k):
  Nn, Nm, Np = n_data[0]

  sub_dim = Nn*Nm*Np

  kx, ky, kz = k

  for s in range(2):
    for n in range(Nn):
      for m in range(Nm):
        for p in range(Np):
          i = get_idx(s,n,m,p,n_data[0])  
          j = get_idx(s,n-1,m,p,n_data[0])
          if 0<=i<=sub_dim and 0<=j<=sub_dim:
            A[i,j] += -1j*kx*(np.sqrt(n/2))

          i = get_idx(s,n,m,p,n_data[0])  
          j = get_idx(s,n+1,m,p,n_data[0])
          if 0<=i<=sub_dim and 0<=j<=sub_dim:
            A[i,j] += -1j*kx*(np.sqrt((n+1)/2))

          i = get_idx(s,n,m,p,n_data[0])  
          j = get_idx(s,n,m-1,p,n_data[0])
          if 0<=i<=sub_dim and 0<=j<=sub_dim:
            A[i,j] += -1j*ky*(np.sqrt(m/2))

          i = get_idx(s,n,m,p,n_data[0])  
          j = get_idx(s,n,m+1,p,n_data[0])
          if 0<=i<=sub_dim and 0<=j<=sub_dim:
            A[i,j] += -1j*ky*(np.sqrt((m+1)/2))

          i = get_idx(s,n,m,p,n_data[0])  
          j = get_idx(s,n,m,p-1,n_data[0])
          if 0<=i<=sub_dim and 0<=j<=sub_dim:
            A[i,j] += -1j*kz*(np.sqrt(p/2))
  
          i = get_idx(s,n,m,p,n_data[0])  
          j = get_idx(s,n,m,p+1,n_data[0])
          if 0<=i<=sub_dim and 0<=j<=sub_dim:
            A[i,j] += -1j*kz*(np.sqrt((p+1)/2))

  return A

### Create $L_2$ operator

In [3]:
def L2(A, n_data, k, qs):
  Nn, Nm, Np = n_data[0]

  kx, ky, kz = k

  qe, qi = qs

  dim_start = 2*Nn*Nm*Np

  args_for_e = ((1, 0, 0, (3, 3, 3)),  # Arguments when e=0
                (0, 1, 0, (3, 3, 3)),  # Arguments when e=1
                (0, 0, 1, (3, 3, 3)))  # Arguments when e=2

  for e in range(3):
    i = dim_start + e
    j = get_idx(0, *args_for_e[e])
    A[i,j] += -qe/np.sqrt(2)
    
    j = get_idx(1, *args_for_e[e])
    A[i,j] += -qi/np.sqrt(2)

  return A


### Create $L_3$ operator

In [4]:
def L3(A, n_data, k):
  Nn, Nm, Np = n_data[0]

  kx, ky, kz = k

  dim_start = 2*Nn*Nm*Np

  # For the electrostatic assumption, L3 is zero

  # Electric field differential terms
  #for e in range(3):
  #  i = dim_start + e
  #  j = dim_start + (e-1)%3 + 3
  #
  #  A[i,j] += 1j*k[(e+1)%3]
  #
  #  j = dim_start + (e+1)%3 + 3
  #  
  #  A[i,j] += -1j*k[(e-1)%3]

  # Magnetic field differential terms (no magnetic field terms due
  # to electrostatic assumption)
  #for b in range(3):
  #  i = dim_start + 3 + b
  #  j = dim_start + (b+1)%3
  #
  #  A[i,j] += 1j*k[(b-1)%3]
  #
  #  j = dim_start + (b-1)%3
  #
  #  A[i,j] += -1j*k[(b+1)%3]

  return A 


### Create $\mathcal{N}$ operator

In [5]:
def N(A, n_data,k):
  Nn, Nm, Np = n_data[0]

  kx, ky, kz = k

  dim_start = 2*Nn*Nm*Np

  for s in range(2):
    for n in range(Nn):
      for m in range(Nm):
        for p in range(Np):
          i = get_idx(s,n,m,p,n_data[0])
          A[i,dim_start:dim_start+3] += 1

  return A

### Construct matrix

In [6]:
L1(A, n_data, k)
L2(A, n_data, k, qs)
L3(A, n_data, k)
N(A, n_data, k)


array([[0.        +0.j        , 0.        -0.70710678j,
        0.        +0.j        , ..., 1.        +0.j        ,
        1.        +0.j        , 1.        +0.j        ],
       [0.        -0.70710678j, 0.        +0.j        ,
        0.        -1.j        , ..., 1.        +0.j        ,
        1.        +0.j        , 1.        +0.j        ],
       [0.        +0.j        , 0.        -1.j        ,
        0.        +0.j        , ..., 1.        +0.j        ,
        1.        +0.j        , 1.        +0.j        ],
       ...,
       [0.        +0.j        , 0.        +0.j        ,
        0.        +0.j        , ..., 0.        +0.j        ,
        0.        +0.j        , 0.        +0.j        ],
       [0.        +0.j        , 0.        +0.j        ,
        0.        +0.j        , ..., 0.        +0.j        ,
        0.        +0.j        , 0.        +0.j        ],
       [0.        +0.j        , 0.70710678+0.j        ,
        0.        +0.j        , ..., 0.        +0.j        ,
 

### Solve initial value problem